In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os

### Load the Data

In [21]:
# Load data and test data
# data = pd.read_csv(r'../Data/train.csv')
# test = pd.read_csv(r'../Data/test.csv')

#relative path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_path = os.path.join(parent_dir, 'Data', 'train.csv')
test_path = os.path.join(parent_dir, 'Data', 'test.csv')

data = pd.read_csv(data_path)
test = pd.read_csv(test_path)

# Separate data into X and Y
y = data.SalePrice
X = data.drop("SalePrice", axis = 1)


### Examine All Features

In [19]:
print("The shape of X is", X.shape)
print("The shape of test set is", test.shape)
print("The columns of X are:\n", X.columns)

The shape of X is (1460, 80)
The shape of test set is (1459, 80)
The columns of X are:
 Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'G

In [20]:
num_name = X.select_dtypes(include=[np.number]).columns
cat_name = X.select_dtypes(include=[np.object_]).columns
print(num_name.shape[0], "numerical variables")
print(cat_name.shape[0], "categorical variables")

37 numerical variables
43 categorical variables


### Data Cleaning and Preprocessing

### Split data into training and validation set

In [42]:
# Split the data into training and validation set
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state=2024)